In [ ]:
import pandas as pd

database = pd.read_csv('/resources/data/dump-20221027/set_de_datos_con_perspectiva_de_genero-database.csv')
validation_fields = pd.read_csv('/resources/data/dump-20221027/set_de_datos_con_perspectiva_de_genero-listado_validacion.csv')
validation_codes = pd.read_csv('/resources/data/dump-20221027/validacion-codigos.csv')
database

In [ ]:
mapper = pd.DataFrame()

In [ ]:
import unidecode
import unicodedata
import re

def strip_accents_spain(string, accents=('COMBINING ACUTE ACCENT', 'COMBINING GRAVE ACCENT')):
    accents = set(map(unicodedata.lookup, accents))
    chars = [c for c in unicodedata.normalize('NFD', string) if c not in accents]
    return unicodedata.normalize('NFC', ''.join(chars))

def normalize(text: str) -> str:
    text = strip_accents_spain(text)
    text = re.sub(r"[_\s]+", " ", text)
    text = text.strip()
    text = re.sub(r" ", "_", text)
    text = text.lower()
    return text


def is_in_public_base(text: str, column: str) -> bool:
    return text in set(database[column])


def is_in_validation_codes(text: str, column: str) -> bool:
    return text in set(validation_codes[column])


def is_in_validation_fields(text: str, column: str) -> bool:
    return text in set(validation_fields[column])


# Conducta

In [ ]:

values = pd.concat([validation_codes['CONDUCTA'], database['CONDUCTA']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

en_base = values.apply(lambda x: is_in_public_base(x, 'CONDUCTA'))
en_val = values.apply(lambda x: is_in_validation_codes(x, 'CONDUCTA'))

norm = values.apply(normalize)
norm

df = pd.DataFrame(
    {
        'CAMPO': 'CONDUCTA',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm,
    }

)

mapper = pd.concat([mapper, df])

# Conducta Descripcion

In [ ]:

values = pd.concat([validation_codes['CONDUCTA_DESCRIPCION'], database['CONDUCTA_DESCRIPCION']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

en_base = values.apply(lambda x: is_in_public_base(x, 'CONDUCTA_DESCRIPCION'))
en_val = values.apply(lambda x: is_in_validation_codes(x, 'CONDUCTA_DESCRIPCION'))

norm = values.apply(normalize)
norm

df = pd.DataFrame(
    {
        'CAMPO': 'CONDUCTA_DESCRIPCION',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm,
    }

)

mapper = pd.concat([mapper, df])

# PERSONA_ACUSADA_NO_DETERMINADA

In [ ]:

values = pd.concat([validation_fields['PERSONA_ACUSADA_NO_DETERMINADA'], database['PERSONA_ACUSADA_NO_DETERMINADA']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'PERSONA_ACUSADA_NO_DETERMINADA'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'PERSONA_ACUSADA_NO_DETERMINADA'))

df = pd.DataFrame(
    {
        'CAMPO': 'PERSONA_ACUSADA_NO_DETERMINADA',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

# df
mapper = pd.concat([mapper, df])

# MODALIDAD_DE_LA_VIOLENCIA

In [ ]:

values = pd.concat([validation_fields['MODALIDAD_DE_LA_VIOLENCIA'], database['MODALIDAD_DE_LA_VIOLENCIA']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'MODALIDAD_DE_LA_VIOLENCIA'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'MODALIDAD_DE_LA_VIOLENCIA'))

df = pd.DataFrame(
    {
        'CAMPO': 'MODALIDAD_DE_LA_VIOLENCIA',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

# df
mapper = pd.concat([mapper, df])

# NIVEL_INSTRUCCION

In [ ]:
values = pd.concat(
    [
        validation_fields["NIVEL_INSTRUCCION"],
        database["NIVEL_INSTRUCCION_ACUSADO/A"],
        database["NIVEL_INSTRUCCION_DENUNCIANTE"],
    ],
    ignore_index=True,
)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(
    lambda x: x
    in set(
        pd.concat(
            [
                database["NIVEL_INSTRUCCION_DENUNCIANTE"],
                database["NIVEL_INSTRUCCION_ACUSADO/A"],
            ]
        )
    )
)

en_val = values.apply(lambda x: is_in_validation_fields(x, "NIVEL_INSTRUCCION"))

df = pd.DataFrame(
    {
        "CAMPO": "NIVEL_INSTRUCCION",
        "ORIGINAL": values,
        "EN_BASE_PUBLICA": en_base,
        "EN_LISTA_VALIDACION": en_val,
        "REQUIERE_NORMALIZACION": norm != values,
        "NORMALIZADO": norm,
    }
)

# df
mapper = pd.concat([mapper, df])


# GENERO

In [ ]:
values = pd.concat(
    [
        validation_fields["GENERO"],
        database["GENERO_ACUSADO/A"],
        database["GENERO_DENUNCIANTE"],
    ],
    ignore_index=True,
)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(
    lambda x: x
    in set(
        pd.concat(
            [
                database["GENERO_DENUNCIANTE"],
                database["GENERO_ACUSADO/A"],
            ]
        )
    )
)

en_val = values.apply(lambda x: is_in_validation_fields(x, "GENERO"))

df = pd.DataFrame(
    {
        "CAMPO": "GENERO",
        "ORIGINAL": values,
        "EN_BASE_PUBLICA": en_base,
        "EN_LISTA_VALIDACION": en_val,
        "REQUIERE_NORMALIZACION": norm != values,
        "NORMALIZADO": norm,
    }
)

# df
mapper = pd.concat([mapper, df])


# FRECUENCIA_EPISODIOS

In [ ]:

values = pd.concat([validation_fields['FRECUENCIA_EPISODIOS'], database['FRECUENCIA_EPISODIOS']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'FRECUENCIA_EPISODIOS'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'FRECUENCIA_EPISODIOS'))

df = pd.DataFrame(
    {
        'CAMPO': 'FRECUENCIA_EPISODIOS',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

# df
mapper = pd.concat([mapper, df])

# RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE

In [ ]:

values = pd.concat([validation_fields['RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE'], database['RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE'))

df = pd.DataFrame(
    {
        'CAMPO': 'RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

# df
mapper = pd.concat([mapper, df])

# ZONA_DEL_HECHO

In [ ]:

values = pd.concat([validation_fields['LUGAR DEL HECHO'], database['ZONA_DEL_HECHO']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

en_base = values.apply(lambda x: is_in_public_base(x, 'ZONA_DEL_HECHO'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'LUGAR DEL HECHO'))

norm = values.apply(normalize)
norm

df = pd.DataFrame(
    {
        'CAMPO': 'ZONA_DEL_HECHO',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm,
    }

)

mapper = pd.concat([mapper, df])

# LUGAR DEL HECHO

In [ ]:

values = pd.concat([validation_fields['VIA_PUBLICA_O_DENTRO_DE_DOMICILIO'], database['LUGAR_DEL_HECHO\n']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

en_base = values.apply(lambda x: is_in_public_base(x, 'LUGAR_DEL_HECHO\n'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'VIA_PUBLICA_O_DENTRO_DE_DOMICILIO'))

norm = values.apply(normalize)
norm

df = pd.DataFrame(
    {
        'CAMPO': 'LUGAR_DEL_HECHO',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm,
    }

)

mapper = pd.concat([mapper, df])

## OBJETO_DE_LA_RESOLUCION

In [ ]:

values = pd.concat([validation_fields['OBJETO_DE_LA_RESOLUCION'], database['OBJETO_DE_LA_RESOLUCION']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'OBJETO_DE_LA_RESOLUCION'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'OBJETO_DE_LA_RESOLUCION'))

df = pd.DataFrame(
    {
        'CAMPO': 'OBJETO_DE_LA_RESOLUCION',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

mapper = pd.concat([mapper, df])

# DETALLE

In [ ]:

values = pd.concat([validation_fields['DETALLE'], database['DETALLE']], ignore_index=True)
values = values.dropna()
values = values.drop_duplicates()
values = values.sort_values()

norm = values.apply(normalize)
norm

en_base = values.apply(lambda x: is_in_public_base(x, 'DETALLE'))
en_val = values.apply(lambda x: is_in_validation_fields(x, 'DETALLE'))

df = pd.DataFrame(
    {
        'CAMPO': 'DETALLE',
        'ORIGINAL': values,
        'EN_BASE_PUBLICA': en_base,
        'EN_LISTA_VALIDACION': en_val,
        'REQUIERE_NORMALIZACION': norm != values,
        'NORMALIZADO': norm
    }
)

mapper = pd.concat([mapper, df])

In [ ]:
mapper = mapper.drop_duplicates()
mapper.to_csv('mapeo-normalizacion-campos.csv', index=False)

In [ ]:

def labelstudio_choice(value: str) -> str:
    value = value.strip()
    return f'<Choice value="{value}" />' 

def is_choice(value: str):
    return value not in ['no_corresponde', 's/d']


In [ ]:
from more_itertools import unique_everseen

choices = mapper.query('CAMPO == "LUGAR_DEL_HECHO"')['NORMALIZADO']
choices = filter(is_choice, choices)
choices = unique_everseen(choices)
choices = map(labelstudio_choice, choices)
choices = list(choices)

for choice in choices:
    print(choice)
# fields